
# 1 Installing and import library

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install datasets
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, TrainingArguments, Seq2SeqTrainingArguments
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 2 Loading model pre-train

In [1]:
! pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")  
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/dataset_summarization/tmp/checkpoint-10000")
# model.to('cuda')

# 3 Preparing datasets

## 3.1 Load datasets

In [3]:
import pandas as pd
dataset = pd.read_csv("/content/drive/MyDrive/dataset_summarization/data_summary.csv")


In [ ]:
dataset

,Unnamed: 0,Summary,Text
0,0,Ngay khi bắt tay ai đó có thể chúng ta đã cảm ...,Tổng thống Mỹ Donald Trump và Tổng thống Pháp ...
1,1,"Rời khỏi Việt Nam vào 10/2016, ông Vũ Đình Duy...",Vũ Đình Duy hồi tháng 5 xuất hiện trước tòa tạ...
2,2,Thủ tướng Iraq xác nhận với BBC rằng Syria khô...,Ông Maliki xác nhận các vụ không kích của Syri...
3,3,Người Việt Nam biết rất ít về ông Trump nhưng ...,Tiến sỹ Vũ Cao Phan tin rằng nếu được tiến hàn...
4,4,Bốn khoa học gia người Mỹ từng đoạt giải Nobel...,GS Jack Steinberger trao đổi với GS Lê Kim Ngọ...
...,...,...,...
102676,102676,"Ép con trẻ học quá sức, mê mẩn thành tích là ...","Đọc bài viết ""Đừng ép con học quá sức"" của tá..."
102677,102677,"Cận kề các kỳ thi chuyển cấp, tốt nghiệp THPT...","Ảnh minh họa. ""Có mẹ là giáo viên chắc là họ..."
102678,102678,"Một loạt các chương trình, khóa học hè online...",Bùng nổ các khóa học hè online Do covid-19 đã...
102679,102679,"""Sự chênh lệch giữa kỳ vọng của bố mẹ với năn...","""Hy vọng con mình học ở những trường top đầu""..."


In [9]:
dataset['Text'][5000]


'Janet Nguyễn phản đối việc vinh danh Tom Hayden Ông Tom Hayden qua đời hồi tháng 10/2016. Bà Janet Nguyễn bị đưa ra khỏi phòng họp Thêm người gốc Việt vào lập pháp California? Bà Janet Nguyễn mở đầu phần phát biểu hôm 23/2 bằng tiếng Việt, sau đó nói bằng tiếng Anh, phản đối việc ông Tom Hayden "ủng hộ Cộng sản Việt Nam". "Tôi và những người con của Chiến sỹ Việt Nam Cộng Hòa sẽ không bao giờ quên sự ủng hộ của cựu Thượng nghị sỹ Tom Hayden cho Cộng sản Việt Nam và sự đàn áp của Cộng sản Việt Nam đối với người dân Việt Nam. "Sau 40 năm, việc làm của các vị như ông đã hại người dân Việt Nam và đã ngăn cản những người Việt Nam vượt biên như gia đình chúng tôi đến đất nước tự do Hoa Kỳ." Chỉ vài chục giây sau khi phát biểu, bà Janet Nguyễn nhiều lần được yêu cầu ngừng và ngồi xuống trước khi bị tắt mic và Thượng nghị sỹ Bill Monning nói bà vi phạm nội quy, đồng thời yêu cầu nhân viên an ninh đưa ra khỏi phòng họp Thượng viện California. Trong cuộc phỏng vấn với Hồng Nga tại văn phòn

In [8]:
print(dataset['Summary'][5000])

Thượng nghị sỹ gốc Việt của tiểu bang California giải thích lý do khiến bà phản đối cố Thượng nghị sỹ Tom Hayden, nhân vật từng tham gia phản đối chiến tranh Việt Nam và là chồng cũ của diễn viên điện ảnh Jane Fonda.


In [ ]:
dataset['Summary'][3000]

'Công ty có lợi nhuận cao nhất thế giới, Apple, thiết lập cấu trúc bí mật mới để giúp tiếp tục tránh trả hàng tỉ đôla tiền thuế, theo Hồ sơ Thiên đường.'

## 3.2 Encode datasets and creating train set data

- Mô hình nhận đầu vào là dữ liệu dạng text sẽ được chuyển về dạng biểu diễn số.

In [ ]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["inputs"], max_length=1024, truncation=True, padding=True
    )
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["labels"], max_length=256, truncation=True, padding=True
        )
    model_inputs['labels'] = labels['input_ids']
    model_inputs['input_ids'] = model_inputs['input_ids']
    return model_inputs

In [ ]:
from torch.utils import data
input_lines = []
label_lines = []

task = 'wikilingua'
train_file = 'train.tsv'

for idx in range(len(dataset)):
  if idx >= 10000:
    break
  input_lines.append(dataset['Text'][idx])
  label_lines.append(dataset['Summary'][idx])


dict_obj = {'inputs': input_lines, 'labels': label_lines}
dataset = Dataset.from_dict(dict_obj)
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=8)


Map (num_proc=8):   0%|          | 0/10000 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3586: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3586: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3586: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your lab

# 4 Fine-tune model

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")


training_args = Seq2SeqTrainingArguments("/content/drive/MyDrive/dataset_summarization/tmp/",
                                      do_train=True,
                                      do_eval=False,
                                      num_train_epochs=1,
                                      learning_rate=1e-5,
                                      warmup_ratio=0.05,
                                      weight_decay=0.01,
                                      per_device_train_batch_size=2,
                                      per_device_eval_batch_size=2,
                                      logging_dir='./log',
                                      group_by_length=True,
                                      save_strategy="epoch",
                                      save_total_limit=3,
                                      #eval_steps=1,
                                      #evaluation_strategy="steps",
                                      # evaluation_strategy="no",
                                      # fp16=True,
                                      )

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
)
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.431800
1000,0.276900
1500,0.237700
2000,0.201500
2500,0.203800
3000,0.239100
3500,0.242300
4000,0.238100
4500,0.251700
5000,0.256600


TrainOutput(global_step=10000, training_loss=0.3002758857727051, metrics={'train_runtime': 3204.7243, 'train_samples_per_second': 3.12, 'train_steps_per_second': 3.12, 'total_flos': 1.21791578112e+16, 'train_loss': 0.3002758857727051, 'epoch': 1.0})

# 5 Evaluate model

In [ ]:
from datasets import load_metric
metric = load_metric("rouge")


<ipython-input-2-ffe5587f2e49>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


# 5.1 Creating test datasets

In [ ]:
import pandas as pd
from torch.utils import data
input_lines = []
label_lines = []

task = 'wikilingua'
train_file = 'train.tsv'
dataset = pd.read_csv("/content/drive/MyDrive/dataset_summarization/data_summary.csv")
for idx in range(10000, len(dataset)):
  if idx >= 11000 :
    break
  input_lines.append(dataset['Text'][idx])
  label_lines.append(dataset['Summary'][idx])



input_lines  = input_lines
label_lines = label_lines
dict_obj = {'inputs': input_lines, 'labels': label_lines}

dataset = Dataset.from_dict(dict_obj)
test_tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=10)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")
     


NameError: ignored

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/dataset_summarization/tmp/checkpoint-10000")
# model.to('cuda')

In [ ]:
import torch 
import numpy as np
metrics = load_metric('rouge')

max_target_length = 256
dataloader = torch.utils.data.DataLoader(test_tokenized_datasets, collate_fn=data_collator, batch_size=32)

predictions = []
references = []
for i, batch in enumerate(tqdm(dataloader)):
  outputs = model.generate(
      # input_ids=batch['input_ids'].to('cuda'),
      input_ids=batch['input_ids'],
      max_length=max_target_length,
      # attention_mask=batch['attention_mask'].to('cuda'),
      attention_mask=batch['attention_mask'],
  )
  with tokenizer.as_target_tokenizer():
    outputs = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in outputs]

    labels = np.where(batch['labels'] != -100,  batch['labels'], tokenizer.pad_token_id)
    actuals = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in labels]
  predictions.extend(outputs)
  references.extend(actuals)
  metrics.add_batch(predictions=outputs, references=actuals)


metrics.compute()

In [ ]:
[{k: v.mid.fmeasure} for k,v in metrics.compute(predictions=predictions, references=references).items()]


# 6 Test some sample

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-large-vietnews-summarization")  
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/dataset_summarization/tmp/checkpoint-10000")
model.cuda()


T5ForConditionalGeneration(
  (shared): Embedding(36096, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36096, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
sentence = """
Lực lượng đoàn viên, thanh niên trong độ tuổi lao động chiếm tỷ lệ khá lớn, nhìn chung lực lượng này trình độ tay nghề của thanh niên còn thấp. Để có nguồn thu nhập, thanh niên xuất khẩu lao động sang các nước như Hàn Quốc, Nhật Bản… Tuy nhiên, một số công ty thu hút lao động hứa hẹn sau khi lao động có thu nhập cao nhưng không đúng như trong hợp đồng, đã có nhiều trường hợp đoàn viên, thanh niên tay trắng trở về và gánh nặng những khoản nợ đã vay để làm các khoản chi phí cho việc xuất khẩu lao động. Đề nghị lãnh đạo các cấp có biện pháp định hướng và cung cấp thông tin từ các doanh nghiệp nhà nước có đủ năng lực đưa lao động đi làm việc tại nước ngoài hợp pháp."""

# Thủ tướng Iraq xác nhận với BBC rằng Syria không kích quân nổi dậy Isis trên lãnh thổ Iraq.

text =  "vietnews: " + sentence + " </s>"
encoding = tokenizer(text, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to("cuda"), encoding["attention_mask"].to("cuda")
outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=256,
    early_stopping=True
)
for output in outputs:
    line = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print(line)

Thanh niên Việt Nam xuất khẩu lao động sang các nước trong khu vực có thu nhập trung bình và thấp đang là lực lượng lao động có thu nhập cao, tuy nhiên chất lượng lao động vẫn chưa cao.
